## Importação dos arquivos e geração dos segmentos

In [3]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO, DIVISÃO (80/20) E SEGMENTAÇÃO (CORRIGIDO)
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---
caminho_raiz = r'CWRU_Bearing_NumPy-main' # IMPORTANTE: Verifique se este caminho está correto
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários de mapeamento
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}

# --- PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3
passo = int(tamanho_segmento * (1 - sobreposicao_percentual))

# --- 2. CARREGAMENTO, DIVISÃO E PROCESSAMENTO ---
dicionario_treino = {} # Dicionário para 80% dos dados normais
dicionario_teste = {} # Dicionário para 20% normais + 100% falhas reais

print(f"Iniciando a leitura e segmentação dos arquivos em '{caminho_raiz}'...")
print("Dados normais serão divididos (80% treino / 20% teste).")
print("Dados de falha real irão 100% para o teste.")

# Função auxiliar para segmentar um sinal e adicionar ao dicionário
def segmentar_e_adicionar(sinal, metadados, dicionario_alvo, chave_base):
    # Verifica se o sinal é longo o suficiente para pelo menos um segmento
    if len(sinal) < tamanho_segmento:
        # print(f"Aviso: Sinal da base '{chave_base}' muito curto ({len(sinal)} amostras) para gerar segmentos. Ignorando.")
        return 0

    num_segmentos_criados = 0
    for i, inicio in enumerate(range(0, len(sinal) - tamanho_segmento + 1, passo)):
        segmento = sinal[inicio : inicio + tamanho_segmento]
        df_segmento = pd.DataFrame({'amplitude': segmento})

        # Adiciona metadados
        df_segmento['arquivo_origem'] = metadados['nome_arquivo']
        df_segmento['rotacao_rpm'] = metadados['rpm']
        df_segmento['tipo_falha'] = metadados['tipo_falha']
        df_segmento['diametro_falha'] = metadados['diametro_falha']
        df_segmento['local_sensor'] = 'Drive End'

        chave_segmento = f"{chave_base}_seg_{i}"
        dicionario_alvo[chave_segmento] = df_segmento
        num_segmentos_criados += 1
    return num_segmentos_criados

# Loop principal pelos arquivos
for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        # Processar apenas arquivos .npz
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)

            # Decodificação de metadados
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm_str = partes[0]
            is_normal = 'Normal' in nome_arquivo

            metadados = {
                'nome_arquivo': nome_arquivo,
                'rpm': int(rpm_str) if rpm_str.isdigit() else 0,
                'tipo_falha': 'Normal' if is_normal else mapa_tipo_falha.get(partes[1].split('@')[0], 'Desconhecido'),
                'diametro_falha': 'N/A' if is_normal else mapa_diametro_falha.get(partes[2], 'Desconhecido')
            }

            try:
                dados_npz = np.load(caminho_completo)
                sensor_cod = 'DE' # Foco apenas no Drive End, como no seu código original

                if sensor_cod in dados_npz.files:
                    sinal_completo = dados_npz[sensor_cod].ravel()

                    if is_normal:
                        # DIVIDE O SINAL NORMAL EM 80/20
                        ponto_corte = int(len(sinal_completo) * 0.8)
                        sinal_treino = sinal_completo[:ponto_corte]
                        sinal_teste = sinal_completo[ponto_corte:]

                        chave_base_normal = f"{nome_sem_ext}_{sensor_cod}"
                        segmentar_e_adicionar(sinal_treino, metadados, dicionario_treino, f"{chave_base_normal}_treino")
                        segmentar_e_adicionar(sinal_teste, metadados, dicionario_teste, f"{chave_base_normal}_teste")

                    else:
                        # Sinais com falha (REAIS) vão inteiramente para o TESTE
                        # Lógica de chave para arquivos de falha (igual ao seu original)
                        partes_chave = nome_sem_ext.split('_')
                        partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                        chave_base_falha = "_".join(partes_chave)
                        
                        # =================================================================
                        # MUDANÇA PRINCIPAL AQUI: Envia falhas reais para o dicionario_teste
                        # =================================================================
                        segmentar_e_adicionar(sinal_completo, metadados, dicionario_teste, chave_base_falha)

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

# --- Relatório Final (Atualizado para refletir a nova lógica) ---
print("\n--- Processo Concluído! ---")
print(f"Total de segmentos de TREINO (APENAS 80% normais): {len(dicionario_treino)}")
print(f"Total de segmentos de TESTE (falhas reais + 20% normais): {len(dicionario_teste)}")

if not dicionario_teste:
    print("\nAVISO: O dicionário de teste está vazio. Verifique se os arquivos 'Normal' existem e se os sinais são longos o suficiente.")

if dicionario_treino:
    # Garante que dicionário não está vazio antes de tentar acessar
    if len(dicionario_treino) > 0:
        chave_exemplo_treino = list(dicionario_treino.keys())[0]
        print(f"\nExemplo de um segmento de TREINO (chave: '{chave_exemplo_treino}'):")
        print(dicionario_treino[chave_exemplo_treino].head())
    else:
        print("\nO dicionário de TREINO está vazio.")

if dicionario_teste:
     # Garante que dicionário não está vazio antes de tentar acessar
    if len(dicionario_teste) > 0:
        chave_exemplo_teste = list(dicionario_teste.keys())[0]
        print(f"\nExemplo de um segmento de TESTE (chave: '{chave_exemplo_teste}'):")
        print(dicionario_teste[chave_exemplo_teste].head())
    else:
        print("\nO dicionário de TESTE está vazio.")

Iniciando a leitura e segmentação dos arquivos em 'CWRU_Bearing_NumPy-main'...
Dados normais serão divididos (80% treino / 20% teste).
Dados de falha real irão 100% para o teste.

--- Processo Concluído! ---
Total de segmentos de TREINO (APENAS 80% normais): 470
Total de segmentos de TESTE (falhas reais + 20% normais): 3184

Exemplo de um segmento de TREINO (chave: '1730_Normal_DE_treino_seg_0'):
   amplitude   arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.014603  1730_Normal.npz         1730     Normal            N/A   
1   0.054449  1730_Normal.npz         1730     Normal            N/A   
2   0.107646  1730_Normal.npz         1730     Normal            N/A   
3   0.133722  1730_Normal.npz         1730     Normal            N/A   
4   0.112652  1730_Normal.npz         1730     Normal            N/A   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  

Exemplo de um segmento de TESTE (chave: '1730_B_14_DE_seg_0'):
 

## Amplitude de falhas modelagem de Tandon

### Outer race

In [1]:
from bearing_utils import calcular_espectro_outer_race
df_outer = calcular_espectro_outer_race(
    fault_diameter_mm=0.1778, 
    rpm=1750, 
    max_harmonics=10
)
print(df_outer.head(15))

   Harmonic_Order  Frequency_Hz  Amplitude_m_s2
0               1    104.563060        0.296435
1               2    209.126120        1.130484
2               3    313.689181        2.344104
3               4    418.252241        3.698679
4               5    522.815301        4.908956
5               6    627.378361        5.684453
6               7    731.941421        5.773090
7               8    836.504481        5.001440
8               9    941.067542        3.306290
9              10   1045.630602        0.753122


### Inner race

In [1]:
from bearing_utils import calcular_espectro_inner_completo

# Use the function
df = calcular_espectro_inner_completo(
    fault_diameter_mm=0.1778, 
    rpm=1750, 
    max_harmonics=10, 
    num_sidebands=3
)
df.head()

,Harmonic_j,Type,Sideband_r,Frequency_Hz,Amplitude_Accel_m_s2
0,1,Sideband Lower,3,70.436940,0.001835
1,1,Sideband Lower,2,99.603606,0.017014
2,1,Sideband Lower,1,128.770273,0.053719
3,1,Main,0,157.936940,0.098195
4,1,Sideband Upper,1,187.103606,0.115251


### Ball bearing 

In [1]:
from bearing_utils import calcular_espectro_ball_completo

df_ball = calcular_espectro_ball_completo(
    fault_diameter_mm=0.1778, 
    rpm=1750, 
    max_harmonics=5,
    num_sidebands=3,
    max_s_iter=50
)

print(df_ball.head(10))

   Harmonic_j            Type  Sideband_r  Frequency_Hz  Amplitude_Accel_m_s2
0           1  Sideband Lower           3    102.660849              0.004181
1           1  Sideband Lower           2    114.278967              0.023772
2           1  Sideband Lower           1    125.897085              0.053990
3           1            Main           0    137.515203              0.049202
4           1  Sideband Upper           1    149.133321              0.075759
5           1  Sideband Upper           2    160.751438              0.047036
6           1  Sideband Upper           3    172.369556              0.011787
7           2  Sideband Lower           3    240.176052              0.028692
8           2  Sideband Lower           2    251.794170              0.144686
9           2  Sideband Lower           1    263.412288              0.296325


## Cálculo dos atributos

## Salva CSV

In [13]:
# =============================================================================
# BLOCO 4: SALVAR OS DATAFRAMES FINAIS EM CSV (COM CAMINHO ESPECÍFICO)
# =============================================================================

import os

# Caminho de saída EXATO fornecido por você
caminho_base_output = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC'

# Cria o diretório se ele não existir
if not os.path.exists(caminho_base_output):
    os.makedirs(caminho_base_output)
    print(f"Diretório criado em: {caminho_base_output}")

# Define os nomes dos arquivos
caminho_csv_treino = os.path.join(caminho_base_output, 'df_treino_features.csv')
caminho_csv_teste = os.path.join(caminho_base_output, 'df_teste_features.csv')

try:
    # Salva o DataFrame de treino
    df_treino.to_csv(caminho_csv_treino, index=False)
    print(f"\nDataFrame de TREINO (features) salvo com sucesso em:")
    print(f"{caminho_csv_treino}")

    # Salva o DataFrame de teste
    df_teste.to_csv(caminho_csv_teste, index=False)
    print(f"\nDataFrame de TESTE (features) salvo com sucesso em:")
    print(f"{caminho_csv_teste}")

except NameError:
    print("\nERRO: Os DataFrames 'df_treino' ou 'df_teste' não foram encontrados.")
    print("Certifique-se de que o Bloco 3 foi executado com sucesso antes de salvar.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado ao salvar os arquivos: {e}")


DataFrame de TREINO (features) salvo com sucesso em:
C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC\df_treino_features.csv

DataFrame de TESTE (features) salvo com sucesso em:
C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset\TCC\df_teste_features.csv
